In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import time
import copy
import os

In [2]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Data Augmentation and Normalization
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Load Datasets
train_dataset = datasets.ImageFolder(root="DL Dataset/train", transform=train_transforms)
val_dataset = datasets.ImageFolder(root="DL Dataset/val", transform=val_transforms)

In [5]:
# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [6]:
# Load Pretrained VGG16 Model
from torchvision import models

model = models.vgg16(pretrained=True)

C:\Users\PRIYANSHU\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\PRIYANSHU\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Modify Fully Connected Layers for Custom Classification
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(train_dataset.classes))

In [8]:
# Move Model to Device (GPU/CPU)
model = model.to(device)

In [9]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
# Early Stopping Parameters
early_stopping_patience = 5
best_val_loss = float("inf")
early_stop_counter = 0

In [11]:
# Training Function
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    global best_val_loss, early_stop_counter

    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training Phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / total

        # Validation Phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct / total

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Early Stopping Check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_counter = 0  # Reset counter when improvement is found
        else:
            early_stop_counter += 1
            print(f"Early stopping counter: {early_stop_counter}/{early_stopping_patience}")

        if early_stop_counter >= early_stopping_patience:
            print("Early stopping triggered!")
            break
 # Load best model weights
    model.load_state_dict(best_model_wts)
    return model 

In [12]:
# Train the Model
trained_model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20)

Epoch 1/20
Train Loss: 1.9765, Train Acc: 0.4208 | Val Loss: 1.5416, Val Acc: 0.5985
Epoch 2/20
Train Loss: 0.8903, Train Acc: 0.7220 | Val Loss: 1.0970, Val Acc: 0.7111
Epoch 3/20
Train Loss: 0.6332, Train Acc: 0.8078 | Val Loss: 1.3687, Val Acc: 0.7227
Early stopping counter: 1/5
Epoch 4/20
Train Loss: 0.4302, Train Acc: 0.8656 | Val Loss: 1.3946, Val Acc: 0.7492
Early stopping counter: 2/5
Epoch 5/20
Train Loss: 0.3183, Train Acc: 0.9015 | Val Loss: 1.4897, Val Acc: 0.7793
Early stopping counter: 3/5
Epoch 6/20
Train Loss: 0.2787, Train Acc: 0.9128 | Val Loss: 1.0946, Val Acc: 0.7945
Epoch 7/20
Train Loss: 0.2086, Train Acc: 0.9374 | Val Loss: 1.1990, Val Acc: 0.7936
Early stopping counter: 1/5
Epoch 8/20
Train Loss: 0.2529, Train Acc: 0.9220 | Val Loss: 1.6105, Val Acc: 0.7802
Early stopping counter: 2/5
Epoch 9/20
Train Loss: 0.1948, Train Acc: 0.9430 | Val Loss: 1.1446, Val Acc: 0.7968
Early stopping counter: 3/5
Epoch 10/20
Train Loss: 0.1633, Train Acc: 0.9479 | Val Loss: 1.929